In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv(r"C:\Users\saake\Downloads\archive\Bank_Personal_Loan_Modelling.csv")

In [4]:
df = pd.get_dummies(df, columns=["Education", "Personal Loan", "Securities Account", "CD Account", "Online", "CreditCard"])

In [5]:
df = df.drop(["ID", "ZIP Code"], axis=1)

In [22]:
class MyNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.log_softmax(self.fc3(x), dim=1)
        return x

In [23]:
def train_model(model, X_train, y_train, epochs, batch_size, lr):
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_X = torch.Tensor(X_train[i:i+batch_size])
            batch_y = torch.LongTensor(y_train[i:i+batch_size])
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")


In [24]:
X_train = [[0.1, 0.2, 0.3], [0.2, 0.3, 0.4], [0.3, 0.4, 0.5], [0.4, 0.5, 0.6]]
y_train = [0, 1, 1, 0]
model = MyNet(input_size=3, hidden_size=4, num_classes=2)
train_model(model, X_train, y_train, epochs=100, batch_size=2, lr=0.01)

Epoch 10/100, Loss: 1.3579
Epoch 20/100, Loss: 1.3313
Epoch 30/100, Loss: 1.2803
Epoch 40/100, Loss: 1.1948
Epoch 50/100, Loss: 1.0518
Epoch 60/100, Loss: 0.8975
Epoch 70/100, Loss: 0.6499
Epoch 80/100, Loss: 0.4037
Epoch 90/100, Loss: 0.2308
Epoch 100/100, Loss: 0.1361


In [25]:
#genetic algorithm


In [26]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy


In [27]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

In [28]:
def fitness(net, X_train, y_train):
    outputs = net(torch.Tensor(X_train))
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == torch.LongTensor(y_train)).sum().item()
    total = len(y_train)
    accuracy = correct / total
    return accuracy

In [29]:
def genetic_algorithm(X_train, y_train, pop_size=10, elite_size=2, mutation_rate=0.1, generations=10):
    input_size = X_train.shape[1]
    output_size = len(np.unique(y_train))
    population = []
    for i in range(pop_size):
        hidden_size = random.randint(2, 100)
        net = Net(input_size, hidden_size, output_size)
        population.append(net)

    for i in range(generations):
        fitness_scores = [(net, fitness(net, X_train, y_train)) for net in population]
        fitness_scores = sorted(fitness_scores, key=lambda x: x[1], reverse=True)
        print("Generation:", i+1, "Best fitness score:", fitness_scores[0][1])
        elites = [fitness_scores[i][0] for i in range(elite_size)]
        new_population = copy.deepcopy(elites)
        while len(new_population) < pop_size:
            parent1 = random.choice(elites)
            parent2 = random.choice(elites)
            child = crossover(parent1, parent2)
            if random.random() < mutation_rate:
                child = mutate(child)
            new_population.append(child)
        population = new_population

    return fitness_scores[0][0]

In [30]:
def crossover(parent1, parent2):
    child = Net(parent1.fc1.in_features, parent1.fc1.out_features, parent1.fc3.out_features)
    child.fc1.weight.data = (parent1.fc1.weight.data + parent2.fc1.weight.data) / 2
    child.fc1.bias.data = (parent1.fc1.bias.data + parent2.fc1.bias.data) / 2
    child.fc2.weight.data = (parent1.fc2.weight.data + parent2.fc2.weight.data) / 2
    child.fc2.bias.data = (parent1.fc2.bias.data + parent2.fc2.bias.data) / 2
    child.fc3.weight.data = (parent1.fc3.weight.data + parent2.fc3.weight.data) / 2
    child.fc3.bias.data = (parent1.fc3.bias.data + parent2.fc3.bias.data) / 2
    return child

In [31]:
def mutate(net):
    child = copy.deepcopy(net)
    layer = random.randint(1, 3)
    if layer == 1:
        child.fc1.weight.data += torch.randn_like(child.fc1.weight.data)

In [32]:
#cultural algorithm

In [33]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [34]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

In [43]:
import random
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [44]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.log_softmax(self.fc3(x), dim=1)
        return x

# define the cultural algorithm
class CulturalAlgorithm:
    def __init__(self, population_size, elite_size, mutation_rate, crossover_rate, num_generations):
        self.population_size = population_size
        self.elite_size = elite_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.num_generations = num_generations
        self.elite_population = []

    def initialize_population(self, net):
        population = []
        for i in range(self.population_size):
            individual = {}
            for name, param in net.named_parameters():
                individual[name] = torch.randn(param.shape)
            population.append(individual)
        return population

    def evaluate_fitness(self, net, individual, X, y):
        with torch.no_grad():
            net.load_state_dict(individual)
            outputs = net(torch.Tensor(X))
            _, predicted = torch.max(outputs.data, 1)
            accuracy = torch.sum(predicted == torch.LongTensor(y)).item() / len(y)
        return accuracy

    def selection(self, population, fitness_scores):
        selected = []
        total_fitness = sum(fitness_scores)
        probabilities = [score/total_fitness for score in fitness_scores]
        for i in range(self.elite_size):
            selected.append(population[np.argmax(fitness_scores)])
            del population[np.argmax(fitness_scores)]
            del fitness_scores[np.argmax(fitness_scores)]
        for i in range(self.population_size - self.elite_size):
            selected.append(random.choices(population, probabilities)[0])
        return selected

    def crossover(self, parent1, parent2):
        child1 = {}
        child2 = {}
        for name in parent1:
            if random.random() < self.crossover_rate:
                child1[name] = parent1[name]
                child2[name] = parent2[name]
            else:
                child1[name] = parent2[name]
                child2[name] = parent1[name]
        return child1, child2

    def mutation(self, individual):
        mutated_individual = {}
        for name in individual:
            if random.random() < self.mutation_rate:
                mutated_individual[name] = individual[name] + torch.randn(individual[name].shape)
            else:
                mutated_individual[name] = individual[name]
        return mutated_individual

    def run(self, X_train, y_train, X_test, y_test):
        # initialize the neural network
        input_size = X_train.shape[1]
        output_size = len(set(y_train))
        hidden_size = 64
        net = Net(input_size, hidden_size, output_size)

        # initialize the population
        population = self.initialize_population(net)

        for generation in range(self.num_generations):
            # evaluate the fitness of each individual in the population
            fitness_scores = []
            for individual in population:
                fitness = self.evaluate_fitness(net, individual, X_train, y_train)
                fitness_scores.append(f

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_20432/2695285494.py, line 89)

In [45]:
#particle swarm optimization

In [46]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [47]:

class PSO:
    def __init__(self, num_particles, max_iters, lr, w, c1, c2):
        self.num_particles = num_particles
        self.max_iters = max_iters
        self.lr = lr
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.swarm_pos = None
        self.swarm_vel = None
        self.p_best_pos = None
        self.p_best_cost = None
        self.g_best_pos = None
        self.g_best_cost = None

    def init_particles(self, num_weights):
        self.swarm_pos = torch.randn(self.num_particles, num_weights)
        self.swarm_vel = torch.randn(self.num_particles, num_weights)
        self.p_best_pos = self.swarm_pos.clone()
        self.p_best_cost = torch.ones(self.num_particles) * np.inf

    def set_fitness_function(self, fitness_fn):
        self.fitness_fn = fitness_fn

    def optimize(self):
        for i in range(self.max_iters):
            # Evaluate fitness of each particle
            particle_costs = self.fitness_fn(self.swarm_pos)
            
            # Update personal best positions
            update_idx = particle_costs < self.p_best_cost
            self.p_best_pos[update_idx] = self.swarm_pos[update_idx]
            self.p_best_cost[update_idx] = particle_costs[update_idx]

            # Update global best position
            g_best_idx = torch.argmin(self.p_best_cost)
            if self.g_best_cost is None or self.p_best_cost[g_best_idx] < self.g_best_cost:
                self.g_best_pos = self.p_best_pos[g_best_idx].clone()
                self.g_best_cost = self.p_best_cost[g_best_idx]

            # Update particle velocities and positions
            r1 = torch.rand(self.num_particles, 1)
            r2 = torch.rand(self.num_particles, 1)
            self.swarm_vel = self.w * self.swarm_vel \
                             + self.c1 * r1 * (self.p_best_pos - self.swarm_pos) \
                             + self.c2 * r2 * (self.g_best_pos - self.swarm_pos)
            self.swarm_pos = self.swarm_pos + self.lr * self.swarm_vel

        return self.g_best_pos, self.g_best_cost

In [49]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define your fitness function
def fitness_fn(weights):
    net.load_state_dict({'fc1.weight': weights[0:12].reshape(4, 3),
                         'fc1.bias': weights[12:15],
                         'fc2.weight': weights[15:19].reshape(1, 4),
                         'fc2.bias': weights[19]})
    with torch.no_grad():
        outputs = net(torch.Tensor(X_train))
        loss = criterion(outputs, torch.LongTensor(y_train))
    return loss.item()

# Initialize PSO optimizer and set fitness function
net = Net(input

TypeError: __init__() missing 2 required positional arguments: 'hidden_size' and 'output_size'